In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *


In [2]:
spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName('MyApp') \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1') \
    .config('spark.mongodb.input.uri', 'mongodb://127.0.0.1/DEP303_ASM1.Questions') \
    .config('spark.mongodb.output.uri', 'mongodb://127.0.0.1/DEP303_ASM1.Questions') \
    .getOrCreate()

:: loading settings :: url = jar:file:/Users/Yamamoto/Spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/Yamamoto/.ivy2/cache
The jars for the packages stored in: /Users/Yamamoto/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e260227d-5e7f-4246-bddb-d29f6e6fd620;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 512ms :: artifacts dl 18ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   art

In [3]:
# schema = StructType([
#     StructField("ID",IntegerType()),
#     StructField("OwnerUserId",IntegerType()),
#     StructField("CreationDate",DateType()),
#     StructField("ClosedDate",DateType()),
#     StructField("Score",IntegerType()),
#     StructField("Body",StringType()),
#     StructField("Body",StringType()),
    
# ])


##	Đọc dữ liệu từ MongoDB

In [7]:
#.format('com.mongodb.spark.sql.DefaultSource')
df = spark.read \
    .format('mongo') \
    .option("database","DEP303_ASM1")\
    .option("collection","Questions")\
    .load()
df.show()
df.printSchema()

+--------------------+--------------------+--------------------+----+-----------+-----+--------------------+--------------------+
|                Body|          ClosedDate|        CreationDate|  Id|OwnerUserId|Score|               Title|                 _id|
+--------------------+--------------------+--------------------+----+-----------+-----+--------------------+--------------------+
|<p>Has anyone got...|                  NA|2008-08-01T15:50:08Z| 120|         83|   21|   ASP.NET Site Maps|{62b72bea57797060...|
|<p>I would like t...|                  NA|2008-08-03T11:12:52Z| 650|        143|   79|Automatically upd...|{62b72bea57797060...|
|<p>I'm trying to ...|                  NA|2008-08-03T20:35:01Z| 810|        233|    9|Visual Studio Set...|{62b72bea57797060...|
|<p>What's the sim...|                  NA|2008-08-04T00:47:25Z| 930|        245|   28|How do I connect ...|{62b72bea57797060...|
|<p>I need to grab...|                  NA|2008-08-04T03:59:42Z|1010|         67|   14|How

## Chuẩn hóa dữ liệu	
Do dữ liệu ở MongoDB được import từ csv nên các trường như CreationDate, ClosedDate sẽ được lưu dưới dạng String chứ không phải Datetime nên bạn sẽ cần chuyển về kiểu dữ liệu DateType(), hoặc có một số giá trị trong trường OwnerUserId có giá trị là "NA", bạn cũng sẽ phải chuyển các giá trị "NA" về null và để kiểu dữ liệu là IntegerType(). Sau khi đọc dữ liệu từ Question thì Dataframe sẽ có Schema như sau:

root
 |-- Id: integer (nullable = true)

 |-- OwnerUserId: integer (nullable = true)

 |-- CreationDate: date (nullable = true)

 |-- ClosedDate: date (nullable = true)

 |-- Score: integer (nullable = true)

 |-- Title: string (nullable = true)

 |-- Body: string (nullable = true)

In [8]:
#một số giá trị trong trường OwnerUserId có giá trị là "NA", bạn cũng sẽ phải chuyển các giá trị "NA" về null và để kiểu dữ liệu là IntegerType()
df.withColumn("OwnerUserId",regexp_replace("OwnerUserId","NA","null"))

transformed_df = df.select(
    col("Id").cast("Integer"),
    col("OwnerUserId").cast("Integer"),
    col("CreationDate").cast("Date"),
    col("ClosedDate").cast("Date"),
    col("Score"),
    col("Title"),
    col("Body")
    )

    
# transformed_df.sort("OwnerUserId").show()
transformed_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- CreationDate: date (nullable = true)
 |-- ClosedDate: date (nullable = true)
 |-- Score: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Body: string (nullable = true)



In [29]:
#Bạn phải 
#transformed_df.createTempView("transformed_table")
#spark.sql("select * from transformed_table where OwneruserId is NULL ").show()

In [30]:
#sử dụng các toán tử IS NULL hoặc IS NOT NULL để kiểm tra một giá trị NULL. Kiểm tra kết quả
transformed_df.select("OwnerUserId").filter("OwnerUserId is NULL").show(3)


+-----------+
|OwnerUserId|
+-----------+
|       null|
|       null|
|       null|
+-----------+
only showing top 3 rows



## 4. Yêu cầu 1: Tính số lần xuất hiện của các ngôn ngữ lập trình
Với yêu cầu này, bạn sẽ cần đếm số lần mà các ngôn ngữ lập trình xuất hiện trong nội dung của các câu hỏi. Các ngôn ngữ lập trình cần kiểm tra là:
Java, Python, C++, C#, Go, Ruby, Javascript, PHP, HTML, CSS, SQL
Để hoàn thành yêu cầu này, bạn có thể sử dụng regex để trích xuất các ngôn ngữ lập trình đã xuất hiện trong từng câu hỏi. Sau đó sử dụng các phép Aggregation để tính tổng theo từng ngôn ngữ. Kết quả sẽ như sau:

In [ ]:
'''
+-------------------+------+                                                    
|Programing Language| Count|
+-------------------+------+
|                 C#| 32414|
|                C++| 28866|
|                CSS| 33556|
|               HTML| 89646|
|                PHP| 63479|
|                SQL|146094|
|                 Go| 79912|
|               Ruby| 16318|
|             Python| 44817|
|               Java|106659|
+-------------------+------+
'''

In [91]:
# lit(str) thêm 1 string vào column  withColumn(col, when().when().when())
#sử dụng regex để trích xuất các ngôn ngữ lập trình đã xuất hiện của cả 2 cột Titlte và cả Body 
after_regrex_df = transformed_df\
    .withColumn("ProgramingLanguage"
            ,when(col("Title").rlike('[Cc]#(?=[?.,\s])'),lit('C#'))   
            .when(col("Title").rlike('[Cc]{1}\+{2}(?=[?.,\s])'),lit('C++'))  
            .when(col("Title").rlike('[Cc]{1}[Ss]{1}[Ss]{1}(?=[?.,\s])'),lit('CSS'))   
            .when(col("Title").rlike('[Hh]{1}[Tt]{1}[Mm]{1}[Ll]{1}(?=[?.,\s])'),'HTML')        
            .when(col("Title").rlike( '[Pp]{1}[Hh]{1}[Pp]{1}(?=[?.,\s])'),'PHP')  
            .when(col("Title").rlike( '[Ss]{1}[Qq]{1}[Ll]{1}(?=[?.,\s])'),'SQL')    
            .when(col("Title").rlike('Go(?=[?.,\s])'),'Go')           
            .when(col("Title").rlike('[Rr]{1}[Uu]{1}[Bb]{1}[Yy]{1}(?=[?.,\s])'),'Ruby')     
            .when(col("Title").rlike('[Pp]{1}[Yy]{1}[Tt]{1}[Hh]{1}[Oo]{1}[Nn]{1}(?=[?.,\s])'),'Python') 
            .when(col("Title").rlike('[Jj]{1}[Aa]{1}[Vv]{1}[Aa]{1}(?=[?.,\s])'),'Java')  
            .when(col("Body").rlike('[Cc]#(?=[?.,\s])'),lit('C#'))   
            .when(col("Body").rlike('[Cc]{1}\+{2}(?=[?.,\s])'),lit('C++'))  
            .when(col("Body").rlike('[Cc]{1}[Ss]{1}[Ss]{1}(?=[?.,\s])'),lit('CSS'))   
            .when(col("Body").rlike('[Hh]{1}[Tt]{1}[Mm]{1}[Ll]{1}(?=[?.,\s])'),'HTML')  
            .when(col("Body").rlike( '[Pp]{1}[Hh]{1}[Pp]{1}(?=[?.,\s])'),'PHP')  
            .when(col("Body").rlike( '[Ss]{1}[Qq]{1}[Ll]{1}(?=[?.,\s])'),'SQL')         
            .when(col("Body").rlike('Go(?=[?.,\s])'),'Go')            
            .when(col("Body").rlike('[Rr]{1}[Uu]{1}[Bb]{1}[Yy]{1}(?=[?.,\s])'),'Ruby')       
            .when(col("Body").rlike('[Pp]{1}[Yy]{1}[Tt]{1}[Hh]{1}[Oo]{1}[Nn]{1}(?=[?.,\s])'),'Python')    
            .when(col("Body").rlike('[Jj]{1}[Aa]{1}[Vv]{1}[Aa]{1}(?=[?.,\s])'),'Java')  
            )
            
         
            

In [93]:
#sử dụng các phép Aggregation để tính tổng theo từng ngôn ngữ
answer1_df = after_regrex_df\
    .where("ProgramingLanguage is Not NULL")\
    .groupBy("ProgramingLanguage")\
    .count()
answer1_df.show()

+------------------+-----+
|ProgramingLanguage|count|
+------------------+-----+
|                C#|33170|
|               C++|23325|
|               CSS|33131|
|              HTML|76034|
|               PHP|73182|
|               SQL|72986|
|                Go| 2077|
|              Ruby|11048|
|            Python|38556|
|              Java|78680|
+------------------+-----+



## 5. Yêu cầu 2 : Tìm các domain được sử dụng nhiều nhất trong các câu hỏi
Trong các câu hỏi thường chúng ta sẽ dẫn link từ các trang web khác vào. Ở yêu cầu này, bạn sẽ cần tìm xem 20 domain nào được người dùng sử dụng nhiều nhất. Chú ý rằng các domain sẽ chỉ gồm tên domain, các bạn sẽ không cần trích xuất những tham số phía sau. Ví dụ về một domain: www.google.com, www.facebook.com,...

Để hoàn thành được yêu cầu này, bạn có thể sử dụng regex để trích xuất các url, sau đó áp dung một số biện pháp xử lý chuỗi để lấy ra được tên của domain, cuối cùng là dùng Aggregation để gộp nhóm lại. Kết quả sẽ như sau:


In [3]:
# +--------------------+-----+                                                    
# |              Domain|Count|
# +--------------------+-----+
# |   www.cs.bham.ac.uk|    4|
# |groups.csail.mit.edu|    7|
# |     fiddlertool.com|    1|
# |   www.dynagraph.org|    1|
# | images.mydomain.com|    1|
# |  img7.imageshack.us|    3|
# +--------------------+-----+

# Các domain này nằm trong Answers Collection.Body nên ta sẽ load Collection vào rồi thực hiện thao tác
Answers_df = spark.read.format("mongo")\
    .option("databse","DEP303_ASM1")\
    .option("collection","Answers")\
    .load()
    
Answers_df.printSchema()


root
 |-- Body: string (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- Id: integer (nullable = true)
 |-- OwnerUserId: string (nullable = true)
 |-- ParentId: integer (nullable = true)
 |-- Score: integer (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)



In [4]:
# # Chia giá trị trong cột Body làm 3 phần, chỉ lấy giá trị sau href= vào dấu / thứ 3.
# trim() remove whitespace from both side
pattern = '(.*)((?<=href=\\")(.*?)\/\/(.*?)(?=\/))(.*)'
Answers_df\
    .select(regexp_extract("Body",pattern,2).alias("Domain"))\
    .where("trim(Domain) != '' ")\
    .groupBy("Domain")\
    .count()\
    .sort(col("count").desc())\
    .show(20,truncate = False)


+-----------------------------+-----+
|Domain                       |count|
+-----------------------------+-----+
|http://stackoverflow.com     |64807|
|https://github.com           |36551|
|http://jsfiddle.net          |35726|
|http://msdn.microsoft.com    |33389|
|http://en.wikipedia.org      |16747|
|http://docs.oracle.com       |11373|
|http://developer.android.com |11078|
|http://php.net               |10772|
|http://api.jquery.com        |8324 |
|https://developer.mozilla.org|8204 |
|https://msdn.microsoft.com   |7320 |
|http://i.stack.imgur.com     |7320 |
|http://code.google.com       |5803 |
|https://jsfiddle.net         |5106 |
|https://developer.apple.com  |5077 |
|http://docs.python.org       |4804 |
|http://dev.mysql.com         |4725 |
|http://developer.apple.com   |4410 |
|https://developers.google.com|4214 |
|https://docs.python.org      |4036 |
+-----------------------------+-----+
only showing top 20 rows



## 6.Yêu cầu 3 : Tính tổng điểm của User theo từng ngày
Bạn cần biết được xem đến ngày nào đó thì User đạt được bao nhiêu điểm. Ví dụ với dữ liệu như sau:

Để hoàn thành yêu cầu này, bạn sẽ cần sử dụng các thao tác Windowing và các thao tác Aggregation, bạn có thể tham khảo bài Bài 9 : Data Aggregations và Join trên Spark. Kết quả sẽ cần được sắp xếp theo trường OwnerUserId và CreationDate.


In [ ]:
'''
+-----------+------------+-----+
|OwnerUserId|CreationDate|Score|
+-----------+------------+-----+
|         26|  2008-08-01|   26|
|         26|  2008-08-01|  144|
|         83|  2008-08-01|   21|
|    	  83|  2008-08-02|   53|
|         26|  2008-08-02|   29|
+-----------+------------+-----+
Thì bạn sẽ có được kết quả:

+-----------+------------+----------+
|OwnerUserId|CreationDate|TotalScore|
+-----------+------------+----------+
|         26|  2008-08-01|       170|
|         26|  2008-08-02|       199|
|         83|  2008-08-01|        21|
|         83|  2008-08-02|        74|
+-----------+------------+----------+

'''

In [10]:
transformed_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- CreationDate: date (nullable = true)
 |-- ClosedDate: date (nullable = true)
 |-- Score: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Body: string (nullable = true)



In [16]:
from pyspark.sql import Window
running_total_window = Window.partitionBy("Id")\
    .orderBy("CreationDate")\
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

transformed_df.withColumn("TotalScore",sum("Score").over(running_total_window)).select("Id","CreationDate","TotalScore").show()

+-----+------------+----------+
|   Id|CreationDate|TotalScore|
+-----+------------+----------+
|  580|  2008-08-02|        21|
| 1160|  2008-08-04|        36|
| 2750|  2008-08-05|         7|
| 3790|  2008-08-06|         3|
| 4230|  2008-08-07|        34|
| 4610|  2008-08-07|        11|
| 5880|  2008-08-08|         6|
| 6110|  2008-08-08|         3|
| 6890|  2008-08-09|         5|
| 7880|  2008-08-11|        15|
| 8950|  2008-08-12|         0|
|10230|  2008-08-13|        12|
|10580|  2008-08-14|        56|
|10680|  2008-08-14|        11|
|11500|  2008-08-14|        24|
|11520|  2008-08-14|         3|
|13060|  2008-08-16|        15|
|13170|  2008-08-16|         6|
|14410|  2008-08-18|        23|
|14570|  2008-08-18|         0|
+-----+------------+----------+
only showing top 20 rows



## 7. Yêu cầu 4: Tính tổng số điểm mà User đạt được trong một khoảng thời gian
Ở yêu cầu này, bạn sẽ cần tính tổng điểm mà User đạt được khi đặt câu hỏi trong một khoảng thời gian. Ví dụ như bạn muốn tính xem từ ngày 01-01-2008 đến 01-01-2009 thì các user đạt được bao nhiêu điểm từ việc đặt câu hỏi. Các khoảng thời gian này sẽ được khai báo trực tiếp trong code, ví dụ như sau:


START = '01-01-2008'
END = '01-01-2009'

Để hoàn thành yêu cầu này, bạn sẽ cần sử dụng filter() để lọc ra các dữ liệu thỏa mãn từ Dataframe, sau đó có thể làm theo yêu cầu 4. Kết quả sẽ cần được sắp xếp theo trường OwnerUserId và CreationDate, ví dụ:

In [ ]:
'''
+-----------+----------+
|OwnerUserId|TotalScore|
+-----------+----------+
|       1580|         5|
|      18051|         2|
|       4101|        11|
|      18866|         6|
|    2376109|         5|
+-----------+----------+
'''

In [27]:
#Yêu cầu 4: Tính tổng số điểm mà User đạt được trong một khoảng thời gian
START = '2008-08-02'
END = '2009-09-01'
transformed_df.filter(col("CreationDate").between(START,END)).groupBy("OwnerUserId").agg(sum("Score").alias("TotalScore")).select("OwnerUserId","TotalScore").show()

+-----------+----------+
|OwnerUserId|TotalScore|
+-----------+----------+
|       1580|        13|
|      18051|         2|
|       4101|        11|
|      18866|         6|
|    2376109|         5|
|      15727|         6|
|       3175|         4|
|      33412|         2|
|      22521|         1|
|      20382|         2|
|       2122|        26|
|      42635|         7|
|      36131|        47|
|      23571|         8|
|      53191|        99|
|      44437|       140|
|      25517|        15|
|      50348|         0|
|      69048|         1|
|        392|       218|
+-----------+----------+
only showing top 20 rows



## 8.Yêu cầu 5: Tìm các câu hỏi có nhiều câu trả lời

Một câu hỏi tốt sẽ được tính số lượng câu trả lời của câu hỏi đó, nếu như câu hỏi có nhiều hơn 5 câu trả lời thì sẽ được tính là tốt. Bạn sẽ cần tìm xem có bao nhiêu câu hỏi đang được tính là tốt,  

Để hoàn thành yêu cầu này, bạn sẽ cần sử dụng các thao tác Join để gộp dữ liệu từ Answers với Collections, sau đó dụng các thao tác Aggregation để gộp nhóm, tính xem mỗi câu hỏi đã có bao nhiêu câu trả lời. Cuối cùng là dùng hàm filter() để lọc ra các câu hỏi có nhiều hơn 5 câu trả lời. 

Lưu ý: Do thao tác có thể tốn rất nhiều thời gian, nên bạn hãy sử dụng cơ chế Bucket Join để phân vùng cho các dữ liệu trước. Bạn có thể tham khảo Bài 9 : Data Aggregations và Join để hiểu rõ hơn về cơ chế này.

Kết quả sẽ cần được sắp xếp theo ID của các câu hỏi

In [37]:
#Yêu cầu 5: Tìm các câu hỏi có nhiều câu trả lời
#Join transformed_df vs Answers_df
join_expression = transformed_df.Id == Answers_df.ParentId
joined_df = transformed_df.join(Answers_df,join_expression,"inner")\
    .select(transformed_df.Id,Answers_df.Id.alias("Answer_ID"))\
    .groupBy(transformed_df.Id)\
    .count()\
    .filter("count >5" )\
    .sort(col("count").desc())
joined_df.show()

+--------+-----+
|      Id|count|
+--------+-----+
|  406760|  408|
|   38210|  316|
|   23930|  129|
|  100420|  100|
|   40480|   69|
|  490420|   67|
|  106340|   61|
| 2155930|   59|
|  226970|   55|
|  202750|   51|
| 1218390|   50|
|17054000|   49|
|   24270|   45|
|  274230|   43|
|  173400|   43|
|  728360|   42|
| 1009160|   41|
|  182630|   40|
| 3412730|   40|
|   64790|   40|
+--------+-----+
only showing top 20 rows



## (Nâng cao)Yêu cầu 6: Tìm các Active User
Một User được tính là Active sẽ cần thỏa mãn một trong các yêu cầu sau:

Có nhiều hơn 50 câu trả lời hoặc tổng số điểm đạt được khi trả lời lớn hơn 500.

Có nhiều hơn 5 câu trả lời ngay trong ngày câu hỏi được tạo.

Bạn hãy lọc các User thỏa mãn điều kiện trên

In [56]:
#Đầu tiên ta Join 2 DF Questions và Answers, lấy ra những cột cần để thao tác Aggregate
join_expression = transformed_df.Id == Answers_df.ParentId
joined_df3 = transformed_df.join(Answers_df,join_expression,"inner").select(Answers_df.OwnerUserId,transformed_df.Id,Answers_df.Id.alias("Answer_Id"),Answers_df.Score,transformed_df.CreationDate,Answers_df.CreationDate.alias("Answer_Date"))
    
joined_df3.show()

+-----------+----+---------+-----+------------+--------------------+
|OwnerUserId|  Id|Answer_Id|Score|CreationDate|         Answer_Date|
+-----------+----+---------+-----+------------+--------------------+
|        149| 580|      585|   13|  2008-08-02|2008-08-02T23:40:04Z|
|         34| 580|      586|   17|  2008-08-02|2008-08-02T23:51:09Z|
|        116| 580|      590|    2|  2008-08-02|2008-08-03T00:22:03Z|
|        111| 580|      591|    3|  2008-08-02|2008-08-03T00:37:03Z|
|         76| 580|      597|    1|  2008-08-02|2008-08-03T01:38:02Z|
|        307| 580|     1446|    2|  2008-08-02|2008-08-04T17:38:59Z|
|         26| 580|     1464|    5|  2008-08-02|2008-08-04T18:00:50Z|
|       1219| 580|     9963|    1|  2008-08-02|2008-08-13T15:41:44Z|
|       1073| 580|    14447|    6|  2008-08-02|2008-08-18T10:47:50Z|
|       1070| 580|    28410|    0|  2008-08-02|2008-08-26T15:38:22Z|
|       1220| 580|    28698|    1|  2008-08-02|2008-08-26T17:39:20Z|
|       3241| 580|    31405|    1|

In [63]:
# Xử lí điều kiện Có nhiều hơn 50 câu trả lời hoặc tổng số điểm đạt được khi trả lời lớn hơn 500.

joined_df4 = joined_df3.groupBy("OwnerUserId").count().filter("count > 50").select("OwnerUserId")

joined_df4.show()

+-----------+
|OwnerUserId|
+-----------+
|      47984|
|      13442|
|     117362|
|      89771|
|     326900|
|     547122|
|      65548|
|     493122|
|     486979|
|    2872922|
|    3661630|
|    4479309|
|      66692|
|     103206|
|     211160|
|     313137|
|     301152|
|     403610|
|     449856|
|     225239|
+-----------+
only showing top 20 rows



In [81]:
#Xử lí điều kiện tổng số điểm đạt được khi trả lời lớn hơn 500.

joined_df5 = joined_df3.select("OwnerUserId","Score").groupBy("OwnerUserId").sum("Score").filter("sum(Score) > 500").select("OwnerUserId")
joined_df5.show()

+-----------+
|OwnerUserId|
+-----------+
|      89771|
|     234938|
|     257550|
|     449856|
|     807231|
|     846273|
|      87234|
|      34211|
|     487339|
|     134633|
|      56829|
|     208809|
|    1705598|
|    1679863|
|      95589|
|     342852|
|     449449|
|     576911|
|       6380|
|       1695|
+-----------+
only showing top 20 rows



In [83]:
# Xử lí điều kiện Có nhiều hơn 5 câu trả lời ngay trong ngày câu hỏi được tạo.
joined_df6 = joined_df3.filter("CreationDate == to_date(Answer_Date)").groupBy("OwnerUserId").count().filter("count > 5").select("OwnerUserId")
joined_df6.show()

+-----------+
|OwnerUserId|
+-----------+
|       5645|
|      30966|
|      47984|
|      76056|
|      13442|
|      31039|
|     142039|
|     115722|
|      12529|
|      89771|
|     102684|
|      70170|
|     234938|
|     192839|
|     114583|
|     344922|
|      80243|
|     326900|
|     247020|
|     312598|
+-----------+
only showing top 20 rows



In [90]:
# Union 3 DF vừa tìm được và loại bỏ các Duplicates ta được DF đáp án

List_ActiveUser_df= joined_df4.union(joined_df5).union(joined_df6).dropDuplicates()
List_ActiveUser_df.show()

+-----------+
|OwnerUserId|
+-----------+
|      47984|
|      13442|
|     117362|
|      89771|
|     326900|
|     547122|
|      65548|
|     493122|
|     486979|
|    2872922|
|    3661630|
|    4479309|
|      66692|
|     103206|
|     211160|
|     313137|
|     301152|
|     403610|
|     449856|
|     225239|
+-----------+
only showing top 20 rows



In [11]:
# people.write.format("mongo").mode("append").option("database",
# "people").option("collection", "contacts").save()